In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from triUtils import *
import open3d as o3d
import pathlib
import scipy.stats as stats
from scipy.special import erf
from math import sqrt
import matplotlib

In [ ]:
folder = '/Users/schimmenti/Desktop/DresdenProjects/wingsurface/vtk_meshes/'
meshes = []
for file in pathlib.Path(folder).glob("*"):
    if(file.is_dir()):
        continue
    if(file.name.endswith('.ply')):
        mesh = o3d.io.read_triangle_mesh(str(file.absolute()))
        verts, tris = np.asarray(mesh.vertices), np.asarray(mesh.triangles)
        meshes.append((verts, tris))

In [ ]:
for idx in range(len(meshes)):
    verts, tris = meshes[idx]

    edgelist = compute_edgelist(tris)

    tri_normals, tri_areas = compute_triangle_normals(verts, tris, return_areas=True)
    vert_normals = compute_vertex_normals(verts, tris)
    vert_barycentric_areas = compute_vertex_barycentric_areas(verts, tris, tri_areas)

    M_taubin = compute_taubin_matrices(verts, tris, vert_normals, tri_areas)
    kappa_1, kappa_2 = compute_taubin_principal_curvatures(M_taubin, vert_normals)
    mean_curvature = -(kappa_1+kappa_2)/2

    cot_lapl_matrix = compute_cot_laplacian(verts, tris, normalize_by_areas=False, return_areas=False)

    %matplotlib inline
    hist, edges = np.histogram(mean_curvature[mean_curvature>0],bins=15,density=True)
    hist, edges = hist[hist>0], edges[:-1][hist>0]
    slopes = []
    for n in range(3, hist.shape[0]):
        slopes.append( stats.linregress(edges[:n],np.log(hist[:n])).slope)
    median_slope = np.median(slopes)
    min_h = -1.0/median_slope
    max_h = 1.00

    clusters = compute_graph_clustering(edgelist,mean_curvature, min_h, max_h,  min_num_points=20 )


    %matplotlib inline
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    #ax.scatter(verts[:,1], verts[:,2], verts[:,0], c=mean_curvature,s=1,cmap='seismic',vmin=-0.1, vmax=0.1)
    ax.scatter(verts[:,1], verts[:,2], verts[:,0], s=0.5,color='black', alpha=0.3)
    for cluster in clusters:
        ax.scatter(verts[cluster,1],verts[cluster,2],verts[cluster,0],s=10)
    ax.set_aspect('equal')
    ax.view_init(elev=90, azim=0)
    plt.show()